## 1. Data Collection

### 1.1 Method
- Source(s) : [rapid api](https://rapidapi.com/)
- Tools used (requests)

### 1.2 Raw data set

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re

In [2]:
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}
base_link = 'https://eg.hatla2ee.com'
all_car_details = []

# Define the exact start and end pages for this script 34an h3mlha 3la agza2
START_PAGE = 1
END_PAGE = 1


def extract_basic_info_from_card(card_soup):
    """Extract basic info from the listing card"""
    info = {}

    # Find the text section with year, mileage, transmission, fuel
    text_section = card_soup.find('div', class_='text-xs flex flex-wrap items-center')
    if text_section:
        spans = text_section.find_all('span')
        text_items = [s.get_text(strip=True) for s in spans if s.get_text(strip=True)]

        if len(text_items) >= 4:
            info['Year'] = text_items[0]
            info['Mileage_in_KM'] = text_items[1].replace(' KM', '').replace(',', '')
            info['Transmission'] = text_items[2]
            info['Fuel_Type'] = text_items[3]

    # Extract price
    price_div = card_soup.find('div', class_='text-lg lg:text-xl font-bold text-primary')
    if price_div:
        price_text = price_div.get_text(strip=True)
        price_match = re.search(r'([\d,]+)', price_text)
        if price_match:
            info['Price'] = price_match.group(1).replace(',', '')

    # Extract make/model
    tag_links = card_soup.find_all('a', class_='inline-flex items-center gap-1 text-gray-500')
    for link in tag_links:
        text = link.get_text(strip=True)
        if link.find('svg'):
            continue
        if 'Make' not in info:
            info['Make'] = text
        elif 'Model' not in info:
            info['Model'] = text
            break

    # Extract city
    city_link = card_soup.find('a', title=lambda t: t and 'For Sale' in t if t else False)
    if city_link:
        info['City'] = city_link.get_text(strip=True)

    return info


#scraaping
for i in range(START_PAGE, END_PAGE + 1):
    print(f"Scraping page {i}  :)")
    url = f'https://eg.hatla2ee.com/en/car/search?page={i}'

    try:
        page_response = requests.get(url, headers=HEADERS, timeout=20)
        page_response.raise_for_status()
        soup = BeautifulSoup(page_response.text, 'html.parser')

        car_containers = soup.find_all('div', attrs={'data-slot': 'card-content'})

        if not car_containers:
            print(f"No cars found on page {i}.")
            break

        print(f"Found {len(car_containers)} cars on page {i} ")

        for car in car_containers:
            try:
                # Extract title from card
                title_span = car.select_one("span.font-semibold.fs-body")
                car_title = title_span.get_text(strip=True) if title_span else 'N/A'
                print(f"getting data: {car_title}")

                # Extract card info
                current_car_details = extract_basic_info_from_card(car)
                current_car_details['Title'] = car_title

                # Find detail page
                link_tag = car.find('a', class_='no-underline')
                if not link_tag:
                    continue

                full_url = base_link + link_tag.get('href')

                # Fetch detail page
                sub_page = None
                for attempt in range(3):
                    try:
                        sub_page_response = requests.get(full_url, headers=HEADERS, timeout=20)
                        sub_page_response.raise_for_status()
                        sub_page = sub_page_response.text
                        break
                    except requests.exceptions.RequestException:
                        time.sleep(attempt * 2 + 1)

                if not sub_page:
                    continue

                detail_soup = BeautifulSoup(sub_page, "html.parser")

                # Extract date
                date_tag = detail_soup.find('div', class_='galleryIconWrap date')
                if date_tag and date_tag.find('span'):
                    current_car_details['Date'] = date_tag.find('span').text.strip()

                # Extract description fields
                desc_items = detail_soup.find_all('div', class_='DescDataItem')
                for item in desc_items:
                    label_tag = item.find('span', class_='DescDataSubTit')
                    value_tag = item.find('span', class_='DescDataVal')
                    if not label_tag or not value_tag:
                        continue

                    label = label_tag.text.strip()
                    value = value_tag.text.strip()

                    if 'Color' in label or 'اللون' in label:
                        current_car_details['Color'] = value
                    elif 'Class' in label or 'الفئة' in label:
                        current_car_details['Class'] = value
                    elif 'Body' in label or 'الشكل' in label:
                        current_car_details['Body_Style'] = value
                    elif 'Cylinder' in label or 'السلندر' in label:
                        current_car_details['Cylinder_Count'] = value
                    elif 'Engine' in label or 'سعة المحرك' in label:
                        current_car_details['Engine_Capacity'] = value

                all_car_details.append(current_car_details)
                print("Successfully scraped")

                time.sleep(random.uniform(1, 3))

            except Exception as e:
                print(f"Error scraping car: {e}")

    except Exception as e:
        print(f"Error loading page {i}: {e}")


print(f"Scraping complete {len(all_car_details)} cars scraped.")


if all_car_details:
    df = pd.DataFrame(all_car_details)

    desired_columns = [
        'Price', 'Date', 'Make', 'Model', 'Year', 'Mileage_in_KM',
        'Transmission', 'City', 'Color', 'Fuel_Type', 'Class',
        'Body_Style', 'Cylinder_Count', 'Engine_Capacity', 'Title'
    ]

    final_columns = [c for c in desired_columns if c in df.columns]
    df = df[final_columns]

    output_file = f'scraped_cars_{START_PAGE}-{END_PAGE}.csv'
    df.to_csv(output_file, index=False, encoding='utf-8-sig')

    print(f"\nSaved to '{output_file}'")
else:
    print("\nNo data scraped.")

Scraping page 1  :)
Found 40 cars on page 1 
getting data: Mercedes GLA 2024
Found 40 cars on page 1 
getting data: Mercedes GLA 2024
Successfully scraped
Successfully scraped
getting data: Mercedes S580 2023
getting data: Mercedes S580 2023
Successfully scraped
Successfully scraped
getting data: Mercedes C 200 2023
getting data: Mercedes C 200 2023
Successfully scraped
Successfully scraped
getting data: Kia Sorento 2024
getting data: Kia Sorento 2024
Successfully scraped
Successfully scraped
getting data: Land Rover Range Rover Sport 2023
getting data: Land Rover Range Rover Sport 2023
Successfully scraped
Successfully scraped
getting data: Mercedes GLC 200 2025
getting data: Mercedes GLC 200 2025
Successfully scraped
Successfully scraped
getting data: Kia Sportage 2026
getting data: Kia Sportage 2026
Successfully scraped
Successfully scraped
getting data: Land Rover Velar 2025
getting data: Land Rover Velar 2025
Successfully scraped
Successfully scraped
getting data: Mercedes Viano 2

## 2. Dataset Overview and Inspection

### <center style="color:#007ACC;">===== BASIC OVERVIEW =====</center>

In [4]:
df = pd.read_csv('scraped_cars_1-100.csv')

In [11]:
print("Shape:", df.shape)
print("="*40)
print("Info:")
print("="*40)
print(df.info())

Shape: (5480, 7)
Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5480 entries, 0 to 5479
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Price          5480 non-null   int64  
 1   Year           5379 non-null   float64
 2   Mileage_in_KM  5379 non-null   float64
 3   Transmission   5379 non-null   object 
 4   City           5480 non-null   object 
 5   Fuel_Type      5379 non-null   object 
 6   Title          5480 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 299.8+ KB
None


### <center style="color:#007ACC;">===== DUPLICATES CHECK =====</center>

In [12]:
#number of duplicates
num_duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

Number of duplicate rows: 778


### <center style="color:#007ACC;">===== Missing Values CHECK =====</center>

In [15]:
# Count missing values per column
num_missing = df_cleaned.isnull().sum()
print("Missing values per column:")
print(num_missing)

Missing values per column:
Price             0
Year             75
Mileage_in_KM    75
Transmission     75
City              0
Fuel_Type        75
Title             0
dtype: int64


## 3. Data Cleaning

### 3.1 Duplicate Removal

In [5]:
df_cleaned = df.drop_duplicates()

In [6]:
num_duplicates = df_cleaned.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

Number of duplicate rows: 0


### 3.2 Handling Missing Values

In [7]:
df_cleaned = df_cleaned.dropna()
num_missing = df_cleaned.isnull().sum()
print("Missing values per column:")
print(num_missing)

Missing values per column:
Price            0
Year             0
Mileage_in_KM    0
Transmission     0
City             0
Fuel_Type        0
Title            0
dtype: int64


### 3.3 Correcting Formats

In [8]:
#printing disinct cities
distinct_cities = df_cleaned['City'].dropna().unique()
print(distinct_cities)

['Mercedes' 'Mitsubishi Eclipse Cross' 'Heliopolis, Cairo' 'Land Rover'
 'Tagamo3 - New Cairo, Cairo' 'Nissan' 'Cairo' 'El Minya'
 'Madinaty, Cairo' 'Warraq, Giza' 'Al Rehab, Tagamo3 - New Cairo, Cairo'
 '5th Settlement' 'Masr Al Jadidah, Cairo' 'El Haram, Giza'
 'Nasr city, Cairo' '10th of Ramadan, Cairo' 'Mitsubishi' 'El Mansoura'
 'Sheikh Zayed City, Giza' '6 October, Giza' 'El Obour, Al Qalyubia'
 'Hyundai' 'Sharqia' 'Jeep' 'Chery' 'Giza' 'Hadayeq El Maadi, Cairo'
 'Alexandria' 'El Agamy, Alexandria' 'Luxor' 'Hurghada, Red Sea'
 'El Mahalla, Gharbia' 'Greater Cairo ( Cairo + Giza)' 'Sheraton, Cairo'
 'Kia' 'Basioun, Gharbia' 'Damietta' 'Cadillac' 'Geely' 'Cupra'
 'Maadi, Cairo' 'Asyut' 'Al Shorouk, Cairo' 'El Katameya, Cairo' 'Opel'
 'Al Marj, Cairo' 'Tanta, Gharbia' 'Faiyum' 'Borg el arab, Alexandria'
 'Obour City, Cairo' 'Haval' 'Seat' 'Baic' 'Renault' 'Ismailia'
 'El Bagour, Monufia' 'Hadaeq Al Qubbah, Cairo'
 'El Zawya El Hamraa, Cairo' 'Kafr El Cheik' 'Dakahlia' 'El-Arish'
 'M

In [9]:
# List of makes/models that should not appear in City (all lowercased when checked)
bad_values = [
    'abarth','arcfox','audi','avatr','bmw','baic','bestune','byd','cadillac',
    'changan','chery','chevrolet','citroën','cupra','deepal','dongfeng','fiat','ford',
    'gac','geely','gmc','haval','hawa','honda','hummer','hyundai','isuzu','jmc','jac',
    'jaguar','jeep','jetour','kyc','kaiyi','kia','lada','land rover','mg','maserati',
    'mercedes','mini','mitsubishi','nissan','opel','peugeot','porsche','proton','renault','rox',
    'seat','shineray','skoda','smart','soueast','ssang yong','suzuki','tesla','toyota','volkswagen','volvo',
    'xiaomi','zeekr','honda crv','hyundai elantra cn7','mitsubishi eclipse cross'
]

# Make a set of normalized bad values for fast lookup
bad_set = {val.lower() for val in bad_values}

def city_contains_bad_value(city_text):
    """Return True if the city_text equals or contains any bad value.
    """
    if not isinstance(city_text, str):
        return True
    normalized = city_text.strip().lower() 
    if not normalized: #for empty string after stripping
        return True
    for bad_value in bad_set:
        # exact match or substring match (e.g., 'audi' in 'audi showroom')
        if normalized == bad_value or bad_value in normalized:
            return True
    return False

# Build a boolean mask using the named function (no lambda)
city_series = df_cleaned['City'].astype(str).fillna('')
mask = city_series.apply(city_contains_bad_value)

# How many rows match and will be changed
num_to_change = int(mask.sum())
print(f"Rows matching listed makes/models in City: {num_to_change}")

# Replace matched City values with 'Unknown'
df_cleaned.loc[mask, 'City'] = 'Unknown'

# Show top City frequencies after replacement
print('\nTop City values (after replacement):')
print(df_cleaned['City'].value_counts().head(20))

Rows matching listed makes/models in City: 717

Top City values (after replacement):
City
Unknown                       717
Tagamo3 - New Cairo, Cairo    546
Cairo                         467
Nasr city, Cairo              325
6 October, Giza               231
Heliopolis, Cairo             220
Alexandria                    219
Sheikh Zayed City, Giza       155
Giza                          137
Maadi, Cairo                  133
El Haram, Giza                 79
Damietta                       53
Obour City, Cairo              46
Al Shorouk, Cairo              43
Mohandessin, Giza              41
Mokattam, Cairo                40
Tanta, Gharbia                 40
Zagazig, Sharqia               38
Madinaty, Cairo                36
Helwan, Cairo                  36
Name: count, dtype: int64


In [10]:
# Standardize City values to canonical governorates (adds `City_standard`)
import re

# Canonical governorates set
governorates = {
    'Cairo','Giza','Alexandria','Red Sea','Aswan','Asyut','Beheira','Beni Suef',
    'Dakahlia','Damietta','Faiyum','Gharbia','Ismailia','Kafr el-Sheikh','Luxor',
    'Matrouh','Minya','Monufia','New Valley','North Sinai','Port Said','Qalyubia',
    'Qena','Sharqia','Sohag','South Sinai','Suez'
}

# Aliases (lowercase keys)
alias = {
    'faiyum': 'Faiyum',
    'el faiyum': 'Faiyum',
    'el wadi el gedid': 'New Valley',
    'menofia': 'Monufia',
    'monufia': 'Monufia',
    'kafr el cheik': 'Kafr el-Sheikh',
    'beheira': 'Beheira',
    'el behiera': 'Beheira',
    'el beheira': 'Beheira',
    'el minya': 'Minya',
    'marsa matrouh': 'Matrouh',
    'marsa matruh': 'Matrouh',
    'sinai': 'North Sinai',
    'greater cairo ( cairo + giza)': 'Cairo',
}

# Precompute lowercase lookups for fast matching
gov_lookup = {g.lower(): g for g in governorates}
alias_lookup = {k.lower(): v for k, v in alias.items()}

def standardize_location(text):
    """Return governorate name for `text` or 'Unknown'.
    - Handles None/non-string safely.
    - Splits text on commas/slashes/hyphens and checks tokens left-to-right.
    - Matches governorate names first, then aliases, then substring match as last resort.
    """
    if text is None:
        return 'Unknown'
    s = str(text).strip()
    if not s or s.lower() == 'unknown':
        return 'Unknown'

    # Split into meaningful tokens (commas, slashes, hyphens are common separators)
    parts = [p.strip() for p in re.split(r'[,/\\-]', s) if p.strip()]

    # 1) exact governorate match (case-insensitive)
    for p in parts:
        key = p.lower()
        if key in gov_lookup:
            return gov_lookup[key]

    # 2) alias lookup
    for p in parts:
        key = p.lower()
        if key in alias_lookup:
            return alias_lookup[key]

    # 3) substring match in joined text (e.g., 'red sea' inside 'hurghada, red sea')
    joined = ' '.join(parts).lower()
    for key, canonical in gov_lookup.items():
        if key in joined:
            return canonical

    return 'Unknown'

# create standardized column
df_cleaned['City_standard'] = df_cleaned['City'].apply(standardize_location)
print('Standardization complete. Top 20 City_standard values:')
print(df_cleaned['City_standard'].value_counts().head(20))

Standardization complete. Top 20 City_standard values:
City_standard
Cairo         2109
Unknown        794
Giza           743
Alexandria     255
Gharbia         90
Sharqia         81
Qalyubia        73
Monufia         69
Dakahlia        65
Damietta        53
Red Sea         38
Beheira         34
Asyut           32
Faiyum          29
Suez            27
Ismailia        23
Minya           21
Sohag           17
Port Said       14
Beni Suef       14
Name: count, dtype: int64


In [11]:
#drop unnecessary columns
df_cleaned = df_cleaned.drop(columns=['City'])
df_cleaned.head()

,Price,Year,Mileage_in_KM,Transmission,Fuel_Type,Title,City_standard
0,3800000,2026.0,0.0,Automatic,Gas,Mercedes C 180 2026,Unknown
1,1475000,2025.0,0.0,Automatic,Gas,Mitsubishi Eclipse Cross 2025,Unknown
2,2550000,2022.0,43000.0,Automatic,Gas,Mercedes C 180 2022,Cairo
3,4100000,2025.0,0.0,Automatic,Gas,Land Rover Range Rover Evoque 2025,Unknown
4,4200000,2026.0,0.0,Automatic,Gas,Mercedes C 200 2026,Unknown


In [12]:
# This removes any 4-digit year (e.g., '2022') from the Title column
df_cleaned['Title'] = df_cleaned['Title'].astype(str).str.replace(r"\b\d{4}\b", "", regex=True).str.strip()
df_cleaned.head()

,Price,Year,Mileage_in_KM,Transmission,Fuel_Type,Title,City_standard
0,3800000,2026.0,0.0,Automatic,Gas,Mercedes C 180,Unknown
1,1475000,2025.0,0.0,Automatic,Gas,Mitsubishi Eclipse Cross,Unknown
2,2550000,2022.0,43000.0,Automatic,Gas,Mercedes C 180,Cairo
3,4100000,2025.0,0.0,Automatic,Gas,Land Rover Range Rover Evoque,Unknown
4,4200000,2026.0,0.0,Automatic,Gas,Mercedes C 200,Unknown


In [49]:
#printing disinct title 
distinct_titles = df_cleaned['Title'].dropna().unique()
print(distinct_titles)

['Mitsubishi Eclipse Cross' 'Mercedes C 180' 'Mercedes C 200'
 'Nissan Sunny' 'Hyundai Accent RB' 'Mercedes GLA 200' 'Skoda Octavia'
 'BMW X1' 'Kia Cerato' 'Renault Logan' 'Lada Oka' 'Peugeot 301'
 'Chevrolet Cruze' 'Opel Astra' 'Mini Cooper' 'Chevrolet Lanos'
 'Mercedes EQA 260' 'Mitsubishi Eclipse' 'Fiat 500' 'Mercedes B 150'
 'Hyundai Elantra' 'Ford Kuga' 'Hyundai Verna' 'Mitsubishi Attrage'
 'Toyota Yaris' 'Hyundai Tucson' 'Volkswagen Passat' 'Subaru XV'
 'Mercedes E 200' 'Chery Arrizo 5' 'Audi Q3' 'Opel Grandland' 'Mazda 2'
 'Nissan Qashqai' 'MG ZS' 'Volkswagen Suran' 'Haval Jolion Pro' 'Audi A5'
 'Chery Tiggo 3' 'Opel Insignia' 'Renault Megane' 'Mazda 3'
 'Citroën C Elysee' 'Kia Picanto' 'Kia Carnival MPV' 'Fiat 132' 'Peugeot'
 'Brilliance Galena' 'Mercedes EQE 350' 'Toyota Corolla' 'Seat Leon'
 'Chery Tiggo 7' 'Geely Emgrand' 'Volkswagen ID 6' 'Cupra Terramar'
 'Speranza A516' 'Changan CS 15' 'MG HS' 'BMW 320' 'Renault Duster'
 'Hyundai Elantra HD' 'Jac J7' 'Fiat 127' 'Skoda Kod

In [13]:
# ========== EXTRACT MAKE & MODEL FROM TITLE ==========
# This handles multi-word brands like "Land Rover", "Alfa Romeo", etc.
import re

# Known brand list (multi-word brands included)
brand_list = [
    "Land Rover", "Alfa Romeo", "Great Wall", "Ssang Yong", "Jetour", "Mini", "DFSK", "MG",
    "BMW", "Mercedes", "Audi", "Volkswagen", "Toyota", "Ford", "Kia", "Hyundai",
    "Renault", "Peugeot", "Nissan", "Mitsubishi", "Fiat", "Chevrolet", "Opel",
    "Skoda", "Volvo", "Subaru", "Changan", "Haval", "Geely", "Chery", "Citroën",
    "Jac", "Daihatsu", "Abarth", "Brilliance", "Proton", "Soueast",
    "Speranza", "Isuzu", "GMC", "Bugatti", "Faw", "Haima", "Saipa", "Kenbo",
    "Senova", "Canghe", "Dongfeng", "Avatr", "Xiaomi", "KYC", "BYD", "Honda",
    "Mazda", "Suzuki", "Lexus", "Jeep", "Dodge", "Chrysler", "Cadillac", "Porsche",
    "Jaguar", "Maserati", "Ferrari", "Lamborghini", "Bentley", "Rolls Royce",
    "Tesla", "Hummer", "Lincoln", "Infiniti", "Acura", "Seat", "Cupra", "Lada",
    "Baic", "Bestune", "GAC", "JMC", "Shineray", "Zeekr", "Deepal", "Arcfox", "Rox"
]

# Sort brands longest-first so multi-word brands match before shorter ones
brand_list_sorted = sorted(brand_list, key=len, reverse=True)

def split_make_model(entry):
    """
    Extract Make and Model from a car title string.
    - First checks against the known brand list (longest match first).
    - Falls back to regex pattern matching for capitalized word sequences.
    - Returns (Make, Model) tuple.
    """
    if not isinstance(entry, str):
        return ("Unknown", None)
    
    entry = entry.strip()
    if not entry:
        return ("Unknown", None)
    
    # 1) Check against known brand list first (case-insensitive, longest match wins)
    entry_lower = entry.lower()
    for brand in brand_list_sorted:
        brand_lower = brand.lower()
        if entry_lower.startswith(brand_lower):
            # Extract the model part (everything after the brand)
            model_part = entry[len(brand):].strip()
            return (brand, model_part if model_part else None)
    
    # 2) Regex fallback: detect sequences of capitalized words as brand
    m = re.match(r"^(([A-Z][a-zA-ZÀ-ÿ]+(?:\s+[A-Z][a-zA-ZÀ-ÿ]+)*))\s*(.*)$", entry)
    if m:
        brand_candidate = m.group(1).strip()
        model_candidate = m.group(3).strip() if m.group(3) else None
        return (brand_candidate, model_candidate)
    
    # 3) No match found
    return ("Unknown", entry)

# Apply to the Title column
df_cleaned[['Make_extracted', 'Model_extracted']] = df_cleaned['Title'].apply(
    lambda x: pd.Series(split_make_model(x))
)

# Show sample results
print("Make/Model extraction complete. Sample results:")
print(df_cleaned[['Title', 'Make_extracted', 'Model_extracted']].head(15))

# Show distribution of extracted makes
print("\nTop 20 extracted Makes:")
print(df_cleaned['Make_extracted'].value_counts().head(20))

Make/Model extraction complete. Sample results:
                            Title Make_extracted     Model_extracted
0                  Mercedes C 180       Mercedes               C 180
1        Mitsubishi Eclipse Cross     Mitsubishi       Eclipse Cross
2                  Mercedes C 180       Mercedes               C 180
3   Land Rover Range Rover Evoque     Land Rover  Range Rover Evoque
4                  Mercedes C 200       Mercedes               C 200
5                  Mercedes C 200       Mercedes               C 200
6                  Mercedes C 180       Mercedes               C 180
7                    Nissan Sunny         Nissan               Sunny
8               Hyundai Accent RB        Hyundai           Accent RB
9                    Nissan Sunny         Nissan               Sunny
10               Mercedes GLA 200       Mercedes             GLA 200
11                  Skoda Octavia          Skoda             Octavia
12                         BMW X1            BMW       

### 3.4 Treating Outliers

In [14]:
#hadlinig outliers in year and prices
df_cleaned = df_cleaned[(df_cleaned['Year'].astype(int) >= 1960) & (df_cleaned['Year'].astype(int) <= 2026)]
price_Q1 = df_cleaned['Price'].astype(float).quantile(0.25)
price_Q3 = df_cleaned['Price'].astype(float).quantile(0.75)
IQR = price_Q3 - price_Q1
df_cleaned = df_cleaned[(df_cleaned['Price'].astype(float) >= (price_Q1 - 1.5 * IQR)) & (df_cleaned['Price'].astype(float) <= (price_Q3 + 1.5 * IQR))]
df_cleaned = df_cleaned.dropna()

In [15]:
df_cleaned.head()

,Price,Year,Mileage_in_KM,Transmission,Fuel_Type,Title,City_standard,Make_extracted,Model_extracted
1,1475000,2025.0,0.0,Automatic,Gas,Mitsubishi Eclipse Cross,Unknown,Mitsubishi,Eclipse Cross
2,2550000,2022.0,43000.0,Automatic,Gas,Mercedes C 180,Cairo,Mercedes,C 180
5,3100000,2022.0,60000.0,Automatic,Gas,Mercedes C 200,Cairo,Mercedes,C 200
6,1190000,2013.0,140000.0,Automatic,Gas,Mercedes C 180,Cairo,Mercedes,C 180
7,775000,2026.0,0.0,Automatic,Gas,Nissan Sunny,Unknown,Nissan,Sunny


In [16]:
df_cleaned.to_csv('cleaned_cars_data.csv', index=False, encoding='utf-8-sig')

## 4. Exploratory Data Analysis (EDA)